In [1]:
# !git clone https://github.com/urchade/GLiNER.git
#%cd GLiNER
!pip install -r requirements.txt
!pip install transformers==4.38.2

# Load examples questions

In [31]:
import pandas as pd
from spacy import displacy
df= pd.read_csv("../Question_catalog.csv")
questions = list(df["Fragen"])
questions

['Wie hat Bayer Leverkusen am 2023-11-25 der Bundesliga-Hinrunde 2023 gespielt?',
 'Welche Teams standen sich am 2023-11-25 in der 1.Bundesliga gegenüber?',
 'Wer hat das erste Tor im Spiel Lerverkusen gegen Bremen am 2023-11-25 geschossen und in welcher Minute?',
 'Gab es im Spiel Leverkusen gegen Bremen am 2023-11-25 einen Platzverweis?',
 'Wie lautet die Halbzeitbilanz in Bezug auf Tore und Ballbesitz im Spiel Leverkusen gegen Bremen in der 1.Bundesliga am 2023-11-25?',
 'Gab es im Spiel Leverkusen gegen Bremen am 2023-11-25 einen Elfmeter?',
 'Wie viele Ecken gab es im Spiel Leverkusen gegen Bremen am 2023-11-25 in der ersten Bundesliga?',
 'Gab es im Spiel Leverkusen gegen Bremen am 2023-11-25 kontroverse Schiedrichterentscheidungen?',
 'Welcher Schiedsrichter hat die Begnung Bremen gegen Leverkusen am 2023-11-25 gepfiffen?',
 'Wie hat Leverkusen gegen Bremen in der Bundesliga-Hinrunde 2023 gespielt?',
 'Wer wurde im Leverkusen gegen Bremen am 2023-11-25 in der 1. Bundesliga ausge

# Initialize GliNER model 

In [22]:
from GLiNER.gliner_ner import GlinerNER

2024-03-19 06:54:50.751579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
model_gli = GlinerNER()
labels = ["date","time", "club", "league"]

# Predict entities for examples questions

In [48]:
for q in questions:
    #print("Question: ", q)
    pred = model_gli.predict_tags(q, labels)
    ent = []
    for p in pred:
        # Time will not required in inference process
        if p["label"] == "time":
            continue
        ent.append(
                {
                    "label": p["label"],
                    "start": p["start"],
                    "end": p["end"]
                }
        )
    ex = [
    {
        "text": q,
        "ents": ent,
        "title": None
    }
    ]
    displacy.render(ex, style="ent", manual=True)
    #print("Predictions: ", pred)
    print("\n-------------\n")


-------------




-------------




-------------




-------------




-------------



...


-------------




-------------




-------------




-------------




-------------



This approach shows its strength and weaknesses. All in all i works well. In post-process we have to pay attention to the processing of synonyms as instance *Leverkusen*, *Bayer*, *Bayer 04*, *Bayer Leverkusen* or *Bundesliga*, *Bundesliga-Hinrunde 2023*, *1.Bundesliga*.

Further, the regonized values have validate, because *Lerverkusen* isn't a league or a club and therefore invalid.

# Date extration out of string

In [47]:
from dateparser.search import search_dates

german_sentences = [
    "Das Meeting ist für den 15. März um 10:00 Uhr angesetzt.",
    "Ihr Geburtstag fällt auf den 4. Juli, und sie plant eine große Party.",
    "Ich habe morgen um 14:30 Uhr einen Termin beim Zahnarzt.",
    "Denken Sie daran, die Sommerzeit beginnt am zweiten Sonntag im März.",
    "Wir landeten am 20. Juli 1969 um 20:17 Uhr UTC auf dem Mond.",
    "Der Laden öffnet am Black Friday, dem 25. November, früh um 5:00 Uhr.",
    "Das Konzert war fantastisch und endete spät, gegen 23:45 Uhr am 9. August.",
    "Unser Jahrestag ist am 14. Februar, und ich habe einen Tisch für 19:00 Uhr reserviert.",
    "Die Sonnenfinsternis wird hier am 8. April sichtbar sein, beginnend um 9:00 Uhr.",
    "Das Feuerwerk zum Neujahr beginnt um Mitternacht am 31. Dezember.",
    "Wie hat Leverkusen am 12. Spieltag der Bundesliga Saison Jahr 2023 und 2024 gespielt?"
]
results = model_gli.predict_tags(german_sentences, labels)
for item in results:
    if item["label"] in ["date"]:
#         
        if search_dates(item["text"]) is None:
            print(item["text"])
            print ('No dates found')
            print("-----------")
        else:
            print(item["text"])
            print(search_dates(item["text"]))
            print("-----------")

15. März
[('15. März', datetime.datetime(2024, 3, 15, 0, 0))]
-----------
4. Juli
[('4. Juli', datetime.datetime(2024, 7, 4, 0, 0))]
-----------
morgen
[('morgen', datetime.datetime(2024, 3, 20, 7, 24, 39, 213052))]
-----------
zweiten Sonntag
[('Sonntag', datetime.datetime(2024, 3, 17, 0, 0))]
-----------
März
[('März', datetime.datetime(2024, 3, 19, 0, 0))]
-----------
20. Juli 1969
[('20. Juli 1969', datetime.datetime(1969, 7, 20, 0, 0))]
-----------
Black Friday
[('Friday', datetime.datetime(2024, 3, 15, 0, 0))]
-----------
25. November
[('November', datetime.datetime(2024, 11, 19, 0, 0))]
-----------
9. August
[('August', datetime.datetime(2024, 8, 19, 0, 0))]
-----------
14. Februar
[('14. Februar', datetime.datetime(2024, 2, 14, 0, 0))]
-----------
8. April
[('April', datetime.datetime(2024, 4, 19, 0, 0))]
-----------
31. Dezember
[('31. Dezember', datetime.datetime(2024, 12, 31, 0, 0))]
-----------
12. Spieltag
No dates found
-----------
Jahr 2023
[('Jahr 2023', datetime.dateti